<h1><center> Méthodes Numériques </center></h1>

In [65]:
from IPython.core.display import display, HTML

def printCenter(content):
    display(HTML(f"<div style='text-align:center'>{content}</div>"))
def printSeparator():
    strg = '======================================================================================================================================'
    print(strg)

class Function:
    def __init__(self, func):
        self.func = func

    ''' Fonction qui approxime une valeur par une méthode itérative pour n fixé ou pour une erreur fixée. '''
    def runApproximation(self, decimal, preciseValue, approxValue: function, delta, forceIterationNb = None):
        if forceIterationNb != None:
                printCenter(f"Nombre d'itérations: {forceIterationNb}")
                printCenter(f"Valeur approchée: {round(approxValue(forceIterationNb), decimal + 4)} ; Valeur exacte: {round(preciseValue, decimal + 4)}")
                printCenter(f"Delta: {round(delta(forceIterationNb), decimal + 4)}")        
        else:
            n = 1
            while delta(n) > 10**(-decimal): n += 1

            printCenter(f"Nombre d'itérations: {n}")
            printCenter(f"Valeur approchée: {round(approxValue(n), decimal + 4)} ; Valeur exacte: {round(preciseValue, decimal + 4)}")
            printCenter(f"Delta: {round(delta(n), decimal + 4)}")   

    ''' Available methods : {"Dichotomy", "FixedPoint", "NewtonRaphson"}'''
    def numericalRootFinder(self, a, b, f, decimal, method, forceIterationNb = None):
        
        if f(a)*f(b) > 0: raise Exception("No root in initial interval.")
        var("x")

        def dichotomyMethod(a, b, f, n):    
            i = 0

            # Tant que l'error est trop grande, on divise les sous intervalles en deux et on choisit l'intervalle qui contient la racine
            while i <= n: 
                middlePoint = (b+a)/2
                i += 1

                if f(a) * f(middlePoint) <= 0:
                    b = middlePoint
                elif f(b) * f(middlePoint) <= 0:
                    a = middlePoint

            # L'intervalle final (a, b) contient la racine et on choisit (a+b)/2 comme approximation
            return numerical_approx((a + b)/2)
        def fixedPointMethod(a, b, f, n):
            # On reformule la recherche des racines de f en terme de recherche de points fixe de g
            g = lambda x : f(x) + x
            # Point initial
            x0 = (b+a)/2
            
            # Constante de contraction de f
            derivative = diff(f(x), x)
            k = find_local_maximum(abs(derivative),a,b)[0]
            if k > 1: raise Exception("Function is not a contraction.")
            
            i = 0
            # Tant que l'erreur est trop grande, on réapplique g à x0
            while i <= n: 
                i += 1
                x0 = g(x0)
            
            return numerical_approx(x0)
        def newtonRaphsonMethod(a, b, f, n):    
            if n > 100: raise Exception("Method did not converge after the maximum number of iterations allowed (100).") 

            # Point initial
            x0 = (b+a)/2
            
            derivative = diff(f(x), x)    
            i = 0
            # Tant que l'erreur est trop grande, on trace la tangente et le nouveau point est l'intersection avec Ox
            while i <= n: 
                i += 1
                x0 = x0 - f(x0)/derivative(x = x0)
            return numerical_approx(x0)
        
        methods = {
            "Dichotomy": dichotomyMethod,
            "FixedPoint": fixedPointMethod,
            "NewtonRaphson": newtonRaphsonMethod,
        }
        
        preciseValue = find_root(f, a, b)
        approxValue = lambda n : methods[method](a, b, f, n)
        delta = lambda n : abs(preciseValue - approxValue(n))

        return self.runApproximation(decimal, preciseValue, approxValue, delta, forceIterationNb)

    ''' Available methods : {"MiddlePoint", "Trapezoid", "Simpson"}'''
    def numericalIntegration(self, a, b, f, decimal, method, forceIterationNb = None):
        middlePoint = lambda f, a, b : (b - a) * f((a + b)/2)
        trapezoid = lambda f, a, b : (b - a) * ((f(a) + f(b))/2)
        simpson = lambda f, a, b : (b-a)*(f(a)+4*f((a + b)/2) + f(b))/6   
        methods = {
            "MiddlePoint": middlePoint,
            "Trapezoid": trapezoid,
            "Simpson": simpson,
        }  

        # Crée une subdivision régulière de [a, b] et approxime l'intégrale par une méthode donnée sur tout les sous-segments
        def compositeMethode(f, a, b, n, method): 
            points = [a + k*(b - a)/n for k in range(n + 1)]
            area = 0
            for i in range(len(points)-1):
                area += method(f, points[i], points[i+1])
            return area
        
        preciseValue = integrate(f(x), x, a,b)

        approxValue = lambda n : compositeMethode(f, a, b, n, methods[method])
        delta = lambda n : abs(preciseValue - approxValue(n))

        return self.runApproximation(decimal, preciseValue, approxValue, delta, forceIterationNb)

    ''' Available methods : {"Newton", "Lagrange"}'''
    def interpolate(self, f, points, method):
        
        def lagrangeBasis(liste):  # Retourne la base de Lagrange associée aux points de L
            basis = []
            var("x")
            for i in range(len(liste)):
                P = 1
                for j in range(len(liste)):
                    if i != j:
                        P *= (x - liste[j]) / (liste[i] - liste[j])
                basis.append(P)
            return basis
        def lagrangeMethod(f, points):
            return sum([f(points[i]) * lagrangeBasis(points)[i] for i in range(len(points))]) 

        def dividedDiff(liste, f):  # Différences divisées récursivement
            n = len(liste)
            if n == 1:
                return f(liste[0])
            else:
                L1 = liste[-n + 1 :]
                L2 = liste[0 : n - 1]
                return (dividedDiff(L1, f) - dividedDiff(L2, f)) / (liste[n - 1] - liste[0])
        def newtonPolynomial(liste):  # On calcule le n-ième polynome de la base de Newton associée aux points de L
            var("x")
            polynomial = 1

            if len(liste) == 0:
                return 1
            for i in range(len(liste)):
                polynomial *= x - liste[i]
            return polynomial
        def newtonMethod(f, points):
            polynomial = 0
            for i in range(1, len(points) + 1):
                coeff = dividedDiff(points[0:i], f)  # On calcule la différence divisée associée à [x0, ..., xi]
                polynomial += coeff * newtonPolynomial(points[0 : i - 1])  # Le polynome est egal a la f[x0, ..., xi] * N_xi
            return polynomial

        methods = {
            "Newton": newtonMethod,
            "Lagrange": lagrangeMethod
        }
        return methods[method](f, points)

decimalPlaces = 5
function = Function(lambda x : sin(x))
p = lambda x : function.interpolate(function.func, [0, 2], "Newton")(x)

#printSeparator()
#printCenter(f"Approximation de la racine dans l'intervalle [{round(0.5,2)}, {round(1,2)}]")
#function.numericalRootFinder(*(0.5, 1), function.func, decimalPlaces, "Dichotomy", 5)
printSeparator()
printCenter(f"Approximation de l'intégrale sur l'intervalle [{round(0,2)}, {round(3.14,2)}]")
function.numericalIntegration(*(0, 3.1415), function.func, decimalPlaces, "Simpson", 4)
printSeparator()


2.00026916994839